# Projeto 1 - Ciência dos Dados

Nome: Rodrigo Carvalho Monteiro de Barros
(sem dupla)

___
Carregando algumas bibliotecas:

In [127]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import operator
import functools
import emoji

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [128]:
import os

filename = 'HBO max.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo HBO max.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [129]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
test = pd.read_excel(filename, sheet_name = 'Teste')

___
## Classificador automático de sentimento


(Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.)

__O produto alvo do trabalho foi o recem lançado serviço de streaming "HBO max". O objetivo é, a partir de tweets que mencionem o produto, obter informações sobre a opinião e aceitação (ou não) dos usuarios (ou possiveis usuarios) desse tipo de serviço, a fim de descobrir se o produto fez sucesso desde seu lançamento e se conseguirá lugar no crescente mercado de serviços de streeming.__ 

Foram considerados relevantes comentarios de opinião sobre o serviço, sejam eles positivos ou negativos, e principalmente comparativos com outros serviços do ramo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Primeiramente foram criadas funções e listas que serão usadas ao longo do codigo. Aqui encontram-se as funções para limpara caracteres especiais e para separar emojis de texto. Também está definido aqui o que foi considerado como *stopwprd* para efeitos desse trabalho.

__Stopword:__ Uma stopword (ou no português:'palavra vazia'), é para a computação o termo usado para denominar uma palavra que é removida antes ou após o processamento de um texto. Não existe uma lista oficial de *stopwords*, pois elas devem se adequar ao contexto em que se faz necessario separa-las, e por isso há necessidade de explicita-las abaixo. No caso desse projeto, foram separadas palavras que nao agregam relevância para a classificação das mensagens, como conectivos, artigos, etc.  

In [130]:
#Função de limpeza que troca alguns sinais básicos por espaços 
def cleanup(text):
    #Retira as sequências de controle
    text = text.replace("\\n", '').replace("\n\n", '').replace("\n", '')
    #Parte "muito simples" retirado do material de aula.
    punctuation = '[!-..:?;_—•|/\‘’“”@#$%&]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    #substitui as palavras "hbo", "max", "rt" e "https" que nao interverem na relevancia da mensagem.
    text_subbed = text_subbed.replace("hbo",'').replace("max",'').replace("rt", '').replace("https", '')
    return text_subbed

#Função que aucilia na limpesa do material analizado, separando emojis de outros caracteres.        
def separa_emoji(lista):
    lista_split_emoji = emoji.get_emoji_regexp().split(lista)
    lista_split_whitespace = [substr.split() for substr in lista_split_emoji]
    lista_split = functools.reduce(operator.concat, lista_split_whitespace)
    return lista_split

#Lista de palavras que nos interessam e serão usadas para comparação mais tarde.    
importantes = ['netflix','amazon','hulu','disney','youtube','sling','fubo','crackle','crunchyroll'\
               ,'vrv','thestreamingwars','streamingwars','streaming' ]    

#lista de 'stopwords' para serem removidas depois:
stopwords = ['the', 'i', 'to', 'on', 'it', 'its', 'is', 'and', 'of', 'a', 'for', 'this', 'in', 'that', 'so', 'an', 'or'\
            , 'with', 'has', 'had', 'at', 'if', 'as', 'by', 'have', 'are', 'get', 'can', 'cant', 'was', 'than', 'but'\
            , 'im', 'be', 'you', 'my', 'me', 'ive', 'been', 'we', 'our']

In [131]:
#separando o dataframe em dois, de acordo com a relevancia feita na coragem.
train_relevantes = train.loc[train.Relevancia == 1]
train_irrelevantes = train.loc[train.Relevancia== 0]

#O codigo não le o dataframe inteiro a menos que ele esteja esposto, se não ele le apenas o "resumo" que aparece no 'print'.
#Estas funções fazem com que o conteúdo por inteiro seja exposto. 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

#================ Relevantes ===============================================================================
#"lista_relevante" é uma enore lista com todos os tweeets relevantes, ja aplicado o cleanup.
lista_relevante = cleanup(str(train_relevantes.Treinamento))
#agora aplica-se a função que separa caracteres colados em emojis
lista_relevante = separa_emoji(lista_relevante.lower())
#"serie_relevante" é um novo dataframe com todas as palavras reparadas.
serie_relevante = pd.Series(lista_relevante)
#criando uma tabela com a contagem da quantidade de cada palavra. Depois, retira as stopwords.
tabela_relevantes = serie_relevante.value_counts()
tabela_relevantes = tabela_relevantes.drop(labels=stopwords)

#================ Irrelevantes =============================================================================
#Agora o mesmo para as irrelevantes.
lista_irrelevante = cleanup(str(train_irrelevantes.Treinamento))
lista_irrelevante = separa_emoji(lista_irrelevante.lower())
serie_irrelevante = pd.Series(lista_irrelevante)
tabela_irrelevantes = serie_irrelevante.value_counts()
tabela_irrelevantes = tabela_irrelevantes.drop(labels=stopwords)

#================ Conjunto completo ========================================================================
#agora o mesmo para o conjunto completo
lista_completa = cleanup(str(train.Treinamento))
lista_completa = separa_emoji(lista_completa.lower())
serie_completa = pd.Series(lista_completa)
tabela_completa = serie_completa.value_counts()
tabela_completa = tabela_completa.drop(labels=stopwords)

#==========================================================================================================
#Calculando as probabilidades da palavra ser relevante ou irrelevante dado o conjunto completo.
P_rel = len(serie_relevante)/len(serie_completa)
P_irrel = 1 - P_rel

In [132]:
#Essa funçao calcula a probabilidade de uma dada palavra estar no grupo relevante e de estar no grupo irrelevante.
def calcula_probabilidades(palavra):

    if palavra in tabela_relevantes:
        R = tabela_relevantes[palavra]
    else:
        R = 0
        
    if palavra in tabela_irrelevantes:
        I = tabela_irrelevantes[palavra]
    else:
        I = 0           
            
    Rel_p = (R + 1) / (len(lista_relevante) + 1*len(train))
    Irrel_p = (I + 1) / (len(lista_irrelevante) + 1*len(train))
    return [Rel_p, Irrel_p] #  Rel_p = P(palavra|Relevantes)  ||  Irrel_p = P(palavra|Irrelevantes)

#Essa funçao usa a anterior para calcular a probabilidade de uma dada frase, por palavra, estar no grupo relevante e
#de estar no grupo irrelevante. Então as compara e retorna a relevância da frase.
def compara_probabilidades(frase):
    frase = separa_emoji(cleanup(str(frase)).lower())
    Rel = 1
    Irrel = 1    
    for palavra in frase:
        Rel *= calcula_probabilidades(palavra)[0]
        Irrel *= calcula_probabilidades(palavra)[1]
    if Rel >= Irrel:
        return 1
    else:
        return 0

In [137]:
#aplicando as funções
lista_teste = []
for e in test['Teste']:
    lista_teste.append(compara_probabilidades(e))
test["classificador"] = lista_teste

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [139]:
#"comparados" é uma tabela comparativa dos valores de relevancia atribuidos manualmente e pelo classificador
comparados = pd.crosstab(test['Relevancia'],test['classificador'], normalize='index')

#separando os valores da tabela 
pos_true = comparados[1][1]/2
pos_false = comparados[1][0]/2
neg_true = comparados[0][0]/2
neg_false =comparados[0][1]/2

print("Verdadeiros positivos: ", round(pos_true*100,2), "%")
print("Falsos positivos: ", round(pos_false*100, 2), "%")
print("Verdadeiros negativos: ", round(neg_true*100,2), "%")
print("Falsos negativos: ", round(neg_false*100, 2), "%")
print("-"*40)

print("Acertos: ", round((pos_true+neg_true)*100,2), "%")
print("Erros: ", round((pos_false+neg_false)*100,2), "%")
print("-"*40)

Verdadeiros positivos:  46.6 %
Falsos positivos:  25.0 %
Verdadeiros negativos:  25.0 %
Falsos negativos:  3.4 %
----------------------------------------
Acertos:  71.6 %
Erros:  28.4 %
----------------------------------------


___
### Concluindo

In [135]:
#criando tabelas separadas com os valores de relevância
test_relevantes = test.loc[test.Relevancia == 1]
test_irrelevantes = test.loc[test.Relevancia== 0]
#================================================================================

print("Verdadeiros positivos: ", round(pos_true*100,2), "%")
print("Falsos positivos: ", round(pos_false*100, 2), "%")
print("Verdadeiros negativos: ", round(neg_true*100,2), "%")
print("Falsos negativos: ", round(neg_false*100, 2), "%")
print("-"*70)

print("Classificados como Relevantes (código): ", round((pos_true+pos_false)*100,2), "%")
print("Classificados como Irelevantes (código): ", round((neg_true+neg_false)*100,2), "%")
print("")
print("Classificados como Relevantes (Manualmente): ", round((len(test_relevantes)/len(test))*100,2), "%")
print("Classificados como Irelevantes (Manualmente): ", round((len(test_irrelevantes)/len(test))*100,2), "%")
print("-"*70)

print("Acertos: ", round((pos_true+neg_true)*100,2), "%    (verdadeiros positivos e verdadeiros negativos)")
print("Erros: ", round((pos_false+neg_false)*100,2), "%    (falsos positivos e falsos negativos)")
print("-"*70)

Verdadeiros positivos:  46.6 %
Falsos positivos:  25.0 %
Verdadeiros negativos:  25.0 %
Falsos negativos:  3.4 %
----------------------------------------------------------------------
Classificados como Relevantes (código):  71.6 %
Classificados como Irelevantes (código):  28.4 %

Classificados como Relevantes (Manualmente):  41.7 %
Classificados como Irelevantes (Manualmente):  58.3 %
----------------------------------------------------------------------
Acertos:  71.6 %    (verdadeiros positivos e verdadeiros negativos)
Erros:  28.4 %    (falsos positivos e falsos negativos)
----------------------------------------------------------------------


***
__Analise de sentimento:__ <br>
A partir dos valores de acertos e erros impressos acima, pode-se concluir que o classificador tem resultados satisfatórios para o que foi proposto, e ajuda bastante a seleção de tweets condizentes com o critério de seleção, de forma que certamente seu uso torna a busca significantemente mais favorável do que sua ausencia. A taxa de falsos negativos foi baixíssima, sendo a taxa de falsos positivos o maior problema do atual código. Uma causa provável do limite de desempenho alcançado pelo classificador é o critério ultilizado para a classificação manual, que pode ter sido vago demais para se obter uma base mais concreta para o classificador. Uma possível solução para esse problema seria obter mais tweets para a base de treinamento do codígo, afim de tornar esse "critério vago" em algo mais claro em termos de quantidade de palavras.
***
__Classificação avançada__ <br>
A fim de melhorar a separação do classificador, foi criada uma segunda função que separa os tweets em quatro categorias: "irrelevantes", "Relevantes", "Muito relevantes" e "Destaque". As novas categorias funcionam da seguinte maneira: <br><br>
*Muito relevantes:* Quando a probabilidade de o tweet ser relevante é significantemente mais provável do que de ser irrelevante.<br><br>
*Destaque:* Quando o tweet apresenta o nome de algum concorrente ou tag que diretamente referencie a comparação entre serviços de Streaming ele ganha destaque imediato nessa categoria, uma vez que o objetivo do projeto é saber sobre o espaço do produto no mercado de trabalho, assumindo que sempre é interessante saber quando o produto for mencionado junto com os concorrentes.

In [136]:
#Essa funçao usa a mesma função "calcula_probabilidades"de antes, para calcular a probabilidade de uma dada frase,
# por palavra, estar no grupo relevante e de estar no grupo irrelevante. Então as compara e retorna a relevância da frase
# de acordo com os novos criterios descritos acima.
def compara_probabilidades_avançado(frase):
    frase = separa_emoji(cleanup(str(frase)).lower())
    Rel = 1
    Irrel = 1    
    for palavra in frase:
        Rel *= calcula_probabilidades(palavra)[0]
        Irrel *= calcula_probabilidades(palavra)[1]
        if palavra in importantes:
            return 3    
    if Rel/Irrel > 1000:
        return 2
    elif Rel >= Irrel:
        return 1
    else:
        return 0

lista_teste_avançado = []
for e in test['Teste']:
    lista_teste_avançado.append(compara_probabilidades_avançado(e))
test["classificador_avançado"] = lista_teste_avançado

Classificador_avançado = test.classificador_avançado.value_counts(normalize=True, sort=False)

print("-"*40)
print("Resultado da Classificação Avançada:")
print("-"*40)
Irrelevantes_A = Classificador_avançado[0]
Relevantes_A = Classificador_avançado[1]
muito_relevantes_A = Classificador_avançado[2]
detaque_A = Classificador_avançado[3]

print("Irrelevantes: ", round(Irrelevantes_A*100,2), "%")
print("Relevantes: ", round(Relevantes_A*100,2), "%")
print("Muito Relevantes: ", round(muito_relevantes_A*100,2), "%")
print("Destaques: ", round(detaque_A*100,2), "%")
print("-"*40)

----------------------------------------
Resultado da Classificação Avançada:
----------------------------------------
Irrelevantes:  27.53 %
Relevantes:  38.06 %
Muito Relevantes:  21.05 %
Destaques:  13.36 %
----------------------------------------


***
__Analise de sentimento do Classificador avançado:__ <br>
O Classificador avançado conseguiu valores satisfatórios para as categorias avançadas, e a proporção de tweets relevantes para muito relevates está equilibrada. A separação dos destaques se provou eficiente também. <br>
É importante resaltar que o mesmo erro de quase 30% do antigo classificador vale ainda para o 'Classificador avançado', assim como as possíveis causas desse erro e as possíveis soluções.

***
__Plano de expanção:__ <br>
Para melhorar ainda mais o código, além de uma maior base de treinamento ou um critério mais claro, pode-se pensar em analisar frases com dupla-negativa ou sarcasmo, funções que o atual código não realiza. Seria necessario analisar a montagem das frases e quantidade de uso de expressões negativas por tweet no primeiro caso, e analisar o significado de emojis e sua coerencia com o conteúdo escrito pode ajudar a identificar ironia e sarcasmo.

***
__Uso do próprio classificador para gerar amostras de treinamento:__ <br>
Não pode-se usar tweets analisados pelo classificador como novas amostras de treinamento para aperfeiçoa-lo. Com uma imprecisão de quase 30%, a nova amostra alimentaria o classificador com erros desde sua origem, de forma que esses se propagariam e levariam a um erro cada vez maior e mais distante do critério de classificalção inicial.

***
__Uso do classificador Naive Bayes em outros contextos:__<br>
Um classificador do tipo Naive Bayes pode ser muito útil em diversos contextos. Por exemplo agora no final do ano um classificador desses poderia ser usado para medir o quanto cada candidato a prefeitura de um estado está sendo comentado nas redes sociais. Um código desses seria mais complexo, tendo que levar em consideração a inclusão digital do público de cada candidato, ou então poderia ser usado para descobrir exatamente esse parâmetro. Esse classificador não poderia ser usado para previsões de voto, mas pode ser um parametro interessante para pensar estratégias de propaganda digital e entender quem tem mais espaço nesse meio, uma vez que nas eleições precidenciais de 2018 muitos disseram que o meio digital foi mais importante meio de propaganda.

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Referencia para função de separação dos Emojis:\
https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python

Referencia para a funçao de limpesa de sequências de controle (e outros):\
https://pt.stackoverflow.com/questions/289388/como-remover-n-de-um-string-em-python